<a href="https://colab.research.google.com/github/CodeNelly-nelson/Finetunning_Medgemma4DR/blob/main/Data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Preprocessing

1. Load bumbledeep/eyepacs
2. Balance to the least-represented class
(undersample)
3. Resize all images to 896×896
4. Split into train/validation/test = 80/10/10 (stratified by class)
5. Push the processed DatasetDict back to Hugging Face




## Install + login

In [ ]:
!pip -q install --upgrade datasets huggingface_hub pillow pyarrow

from huggingface_hub import notebook_login
notebook_login()  # needs a WRITE token to push datasets/models


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 533.4/533.4 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 141.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 57.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.50.0 requires pillow<12.0,>=8.0, but you have pillow 12.1.0 which is incompatible.
transformers 4.57.3 requires huggingface-hub<1.0,>=0.34.0, but you have huggingface-hub 1.3.1 which is incompatible.


## Load bumbledeep/eyepacs and inspect labels

The dataset card describes it as a subset of the Kaggle EyePACS training images.

In [ ]:
from datasets import load_dataset
from collections import Counter

raw = load_dataset("bumbledeep/eyepacs")  # train-only
ds = raw["train"]

print(ds)
print(ds.column_names)

# pick your class column: label_code is usually the cleanest for stratify/balance
label_col = "label_code"  # change if needed

counts = Counter(ds[label_col])
print("Class counts:", dict(counts))
min_count = min(counts.values())
print("Least represented class count:", min_count)


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00014.parquet:   0%|          | 0.00/465M [00:00<?, ?B/s]

data/train-00001-of-00014.parquet:   0%|          | 0.00/467M [00:00<?, ?B/s]

data/train-00002-of-00014.parquet:   0%|          | 0.00/466M [00:00<?, ?B/s]

data/train-00003-of-00014.parquet:   0%|          | 0.00/465M [00:00<?, ?B/s]

...

data/train-00011-of-00014.parquet:   0%|          | 0.00/464M [00:00<?, ?B/s]

data/train-00012-of-00014.parquet:   0%|          | 0.00/466M [00:00<?, ?B/s]

data/train-00013-of-00014.parquet:   0%|          | 0.00/469M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/35108 [00:00<?, ? examples/s]

Dataset({
    features: ['image', 'label_code', 'label'],
    num_rows: 35108
})
['image', 'label_code', 'label']
Class counts: {0: 25802, 1: 2438, 2: 5288, 4: 708, 3: 872}
Least represented class count: 708


## Balance to the least-represented class (undersample)

In [ ]:
import random

seed = 42
random.seed(seed)

# Collect indices by class
indices_by_class = {}
for i, y in enumerate(ds[label_col]):
    indices_by_class.setdefault(y, []).append(i)

# Undersample each class to min_count
balanced_indices = []
for y, idxs in indices_by_class.items():
    random.shuffle(idxs)
    balanced_indices.extend(idxs[:min_count])

random.shuffle(balanced_indices)

balanced = ds.select(balanced_indices)
print(balanced)

from collections import Counter
print("Balanced counts:", dict(Counter(balanced[label_col])))


Dataset({
    features: ['image', 'label_code', 'label'],
    num_rows: 3540
})
Balanced counts: {2: 708, 0: 708, 4: 708, 1: 708, 3: 708}


## Resize images to 896×896 (with PIL)

In [ ]:
from PIL import Image

TARGET_SIZE = (896, 896)

def resize_example(example):
    img = example["image"]
    if not isinstance(img, Image.Image):
        # HF Image feature usually decodes to PIL already, but just in case
        img = Image.open(img).convert("RGB")
    else:
        img = img.convert("RGB")
    example["image"] = img.resize(TARGET_SIZE, resample=Image.BICUBIC)
    return example

# Use a modest num_proc if your runtime has CPU cores
balanced_resized = balanced.map(resize_example, num_proc=2)
print(balanced_resized)


Map (num_proc=2):   0%|          | 0/3540 [00:00<?, ? examples/s]

Dataset({
    features: ['image', 'label_code', 'label'],
    num_rows: 3540
})


## Stratified split 80/10/10 (train/val/test)

In [ ]:
# First split: train 80%, temp 20%
split1 = balanced_resized.train_test_split(
    test_size=0.2,
    seed=seed,
    stratify_by_column=label_col
)

# Second split: temp -> val 10%, test 10% (split temp 50/50)
split2 = split1["test"].train_test_split(
    test_size=0.5,
    seed=seed,
    stratify_by_column=label_col
)

dataset_dict = {
    "train": split1["train"],
    "validation": split2["train"],
    "test": split2["test"]
}

from datasets import DatasetDict
final_ds = DatasetDict(dataset_dict)

final_ds


ValueError: Stratifying by column is only supported for ClassLabel column, and column label_code is Value.

## Quick sanity-check distributions:

In [ ]:
from collections import Counter

for k in ["train","validation","test"]:
    c = Counter(final_ds[k][label_col])
    print(k, dict(c))



## Push the processed dataset to Hugging Face Hub


In [ ]:
HF_DATASET_REPO = "nelsonsoh8/eyepacs-dr-balanced-896"  # <-- change this

final_ds.push_to_hub(HF_DATASET_REPO)
print("Pushed to:", HF_DATASET_REPO)
